<a href="https://colab.research.google.com/github/pandemic-tracking/global-vaccine/blob/main/OWID_WHO_vaccine_administration_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OWID/WHO comparison

In [16]:
import pandas as pd
from datetime import datetime
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [17]:
def print_bold(input):
	print('\033[1m{:10s}\033[0m'.format(input))

# Prepare dataset

In [18]:
# get OWID vaccination timeseries from Github
owid_data = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv")

# drop non cumulative columns from OWID data
owid_data.drop(columns=[
    'daily_vaccinations','total_vaccinations_per_hundred','people_vaccinated_per_hundred','people_fully_vaccinated_per_hundred','total_boosters_per_hundred','daily_vaccinations_per_million',
    'daily_people_vaccinated','daily_people_vaccinated_per_hundred','daily_vaccinations_raw'], inplace = True)

# forward fill empty values in owid dataset
owid_data[['total_vaccinations','people_vaccinated','total_boosters']] = owid_data.groupby('iso_code')[['total_vaccinations','people_vaccinated','total_boosters']].apply(lambda x: x.fillna(method='ffill'))


# get latest date for each country in OWID dataset
latest_owid_dates = owid_data.groupby('iso_code')['date'].max().to_frame()

# merge full date 
latest_owid_data = pd.merge(latest_owid_dates,owid_data,on=['iso_code','date'])

# get WHO data
who_data = pd.read_csv("https://covid19.who.int/who-data/vaccination-data.csv")

# get PTC owid source classifications
owid_sources = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vTDKyIaQVtTIy7kn5pD2W8oKM3YoX3YOdSsH3q-r0INH2axjQl6YxgDHBi4HikKx_cmRElde_E-2vlr/pub?gid=2040574494&single=true&output=csv").filter(['Code','OWID Vax Source Category'])

# merge latest OWID data with WHO data
merged_data = pd.merge(latest_owid_data,who_data, how='inner', left_on='iso_code', right_on='ISO3').drop(columns=[
    'WHO_REGION','TOTAL_VACCINATIONS_PER100', 'PERSONS_VACCINATED_1PLUS_DOSE_PER100','PERSONS_FULLY_VACCINATED_PER100',
    'VACCINES_USED', 'FIRST_VACCINE_DATE', 'NUMBER_VACCINES_TYPES_USED', 'PERSONS_BOOSTER_ADD_DOSE_PER100',
    'PERSONS_BOOSTER_ADD_DOSE','PERSONS_VACCINATED_1PLUS_DOSE','PERSONS_FULLY_VACCINATED','people_vaccinated'
    ])
merged_data.rename(columns = {'date':'owid_date','DATE_UPDATED':'WHO_DATE',
'total_vaccinations':'owid_total_vaccinations',
'people_vaccinated':'owid_people_vaccinated',
'TOTAL_VACCINATIONS':'WHO_TOTAL_VACCINATIONS',
'PERSONS_VACCINATED_1PLUS_DOSE':'WHO_PERSONS_VACCINATED_1PLUS_DOSE',
'PERSONS_FULLY_VACCINATED':'WHO_PERSONS_FULLY_VACCINATED',
}, inplace = True)

# merge combined OWID+WHO data source with PTC owid source classifications
merged_data = pd.merge(merged_data,owid_sources, how='inner', left_on='ISO3', right_on='Code')

# calculate total vaccines diff
merged_data['diff_total_vaccinations'] = merged_data.WHO_TOTAL_VACCINATIONS - merged_data.owid_total_vaccinations
print_bold('Diff = WHO - owid' )
merged_data





Diff = WHO - owid


,iso_code,owid_date,location,owid_total_vaccinations,people_fully_vaccinated,total_boosters,COUNTRY,ISO3,DATA_SOURCE,WHO_DATE,WHO_TOTAL_VACCINATIONS,Code,OWID Vax Source Category,diff_total_vaccinations
0,ABW,2022-09-30,Aruba,173442.0,83617.0,NaN,Aruba,ABW,REPORTING,2022-09-16,173306.0,ABW,Country,-136.0
1,AFG,2022-09-28,Afghanistan,11881035.0,10245183.0,NaN,Afghanistan,AFG,REPORTING,2022-09-28,11881035.0,AFG,World Health Organization,0.0
2,AGO,2022-09-04,Angola,22297104.0,7814121.0,1115824.0,Angola,AGO,REPORTING,2022-09-04,22297104.0,AGO,World Health Organization,0.0
3,AIA,2022-09-30,Anguilla,24408.0,10342.0,3216.0,Anguilla,AIA,REPORTING,2022-09-16,24102.0,AIA,Pan American Health Organization,-306.0
4,ALB,2022-09-18,Albania,2974247.0,1262982.0,356077.0,Albania,ALB,REPORTING,2022-09-18,2974247.0,ALB,World Health Organization,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,WSM,2022-09-26,Samoa,448009.0,177489.0,78510.0,Samoa,WSM,REPORTING,2022-09-12,510313.0,WSM,Pacific Data Hub (PDH),62304.0
210,YEM,2022-09-28,Yemen,1065395.0,604202.0,33507.0,Yemen,YEM,REPORTING,2022-09-28,1065395.0,YEM,World Health Organization,0.0
211,ZAF,2022-09-30,South Africa,37641904.0,19401052.0,3738814.0,South Africa,ZAF,REPORTING,2022-09-18,37580608.0,ZAF,Country,-61296.0
212,ZMB,2022-09-28,Zambia,9408734.0,6096991.0,33684.0,Zambia,ZMB,REPORTING,2022-09-04,9015082.0,ZMB,Country,-393652.0


# Using the merged dataset 

In [19]:
print_bold('\nTotal countries')
print(len( merged_data))

print_bold('\nDates')
dates_matching = len( merged_data.loc[merged_data['owid_date'] == merged_data['WHO_DATE']])
dates_owid_greater = len(merged_data.loc[merged_data['owid_date'] > merged_data['WHO_DATE']])
dates_owid_lesser = len(merged_data.loc[merged_data['owid_date'] < merged_data['WHO_DATE']])
print('Matching dates:' , dates_matching, 'countries')
print('owid greater ' , dates_owid_greater)
print('owid lesser ' , dates_owid_lesser)


matching_total_vaccinations_df = merged_data.loc[merged_data['owid_total_vaccinations'] == merged_data['WHO_TOTAL_VACCINATIONS']]
totalvax_matching = len( matching_total_vaccinations_df.index)
total_matching_vax_who = int(matching_total_vaccinations_df['WHO_TOTAL_VACCINATIONS'].sum(axis=0))
total_matching_vax_owid = int(matching_total_vaccinations_df['owid_total_vaccinations'].sum(axis=0))
print_bold('\nMatching total vaccinations: ' + str(totalvax_matching) + ' countries')
print('WHO total doses: %s' % "{:,}".format(total_matching_vax_who))
print('OWID total doses: %s' % "{:,}".format(total_matching_vax_owid))

# owid greater

owid_greater_total_vaccinations_df = merged_data.loc[merged_data['owid_total_vaccinations'] > merged_data['WHO_TOTAL_VACCINATIONS']]
owid_greater_count = len(owid_greater_total_vaccinations_df)
owid_greater_who_totalvax = int(owid_greater_total_vaccinations_df['WHO_TOTAL_VACCINATIONS'].sum(axis=0))
owid_greater_owid_totalvax = int(owid_greater_total_vaccinations_df['owid_total_vaccinations'].sum(axis=0))

print_bold('\nOWID greater total doses: ' + str(owid_greater_count) +  ' countries')
print('WHO total doses: %s' % "{:,}".format(owid_greater_who_totalvax))
print('OWID total doses: %s' % "{:,}".format(owid_greater_owid_totalvax))
print('DIFF: %s' % "{:,}".format(owid_greater_owid_totalvax-owid_greater_who_totalvax))

# owid lesser
owid_lesser_total_vaccinations_df = merged_data.loc[merged_data['owid_total_vaccinations'] < merged_data['WHO_TOTAL_VACCINATIONS']]
owid_lesser_count = len(owid_lesser_total_vaccinations_df)
owid_lesser_who_totalvax = int(owid_lesser_total_vaccinations_df['WHO_TOTAL_VACCINATIONS'].sum(axis=0))
owid_lesser_owid_totalvax = int(owid_lesser_total_vaccinations_df['owid_total_vaccinations'].sum(axis=0))

print_bold('\nOWID lesser total doses: '  +  str(owid_lesser_count) +  ' countries')
print('WHO total doses: %s' % "{:,}".format(owid_lesser_who_totalvax))
print('OWID total doses: %s' % "{:,}".format(owid_lesser_owid_totalvax))
print('DIFF: %s' % "{:,}".format(owid_lesser_who_totalvax-owid_lesser_owid_totalvax))


print_bold('\nTotal Doses')
totalvax_who = int(merged_data['WHO_TOTAL_VACCINATIONS'].sum(axis=0))
total_vax_owid= int(merged_data['owid_total_vaccinations'].sum(axis=0))
total_vax_diff = totalvax_who-total_vax_owid
print('WHO total doses: %s' % "{:,}".format(totalvax_who))
print('OWID total doses: %s' % "{:,}".format(total_vax_owid))
print('Overall diff total doses (WHO-Owid): %s' % "{:,}".format(total_vax_diff))

# dictionary of lists
dict = {
        'Date' : [pd.Timestamp.now(tz = 'US/Eastern')],
        'Total countries':[len( merged_data)],
        'Dates - matching': [dates_matching],
        'Dates - OWID greater':[dates_owid_greater],
        'Dates - OWID lesser':[dates_owid_lesser],
        'Total Vax - matching': [totalvax_matching],
        'Total Vax - matching- WHO total':[total_matching_vax_who],
        'Total Vax - matching- OWID total':[total_matching_vax_owid],
        'Total Vax - OWID greater - count':[owid_greater_count],
        'Total Vax - OWID greater - WHO total':[owid_greater_who_totalvax],
        'Total Vax - OWID greater - OWID total':[owid_greater_owid_totalvax],
        'Total Vax - OWID greater - Diff':[owid_greater_owid_totalvax-owid_greater_who_totalvax],
        'Total Vax - OWID lesser - count':[owid_lesser_count],
        'Total Vax - OWID lesser - WHO total':[owid_lesser_who_totalvax],
        'Total Vax - OWID lesser - OWID total':[owid_lesser_owid_totalvax],
        'Total Vax - OWID lesser - Diff':[owid_lesser_who_totalvax-owid_lesser_owid_totalvax],
        'Total Vax - WHO total':[totalvax_who],
        'Total Vax - OWID total':[total_vax_owid],
        'Total Vax - Diff':[total_vax_diff],

        }

 
df = pd.DataFrame(dict)
 
df



Total countries
214

Dates    
Matching dates: 103 countries
owid greater  82
owid lesser  29

Matching total vaccinations: 110 countries
WHO total doses: 2,586,464,480
OWID total doses: 2,586,464,480

OWID greater total doses: 73 countries
WHO total doses: 5,137,235,433
OWID total doses: 5,202,839,257
DIFF: 65,603,824

OWID lesser total doses: 31 countries
WHO total doses: 4,942,401,242
OWID total doses: 4,875,695,027
DIFF: 66,706,215

Total Doses
WHO total doses: 12,666,101,155
OWID total doses: 12,664,998,764
Overall diff total doses (WHO-Owid): 1,102,391


,Date,Total countries,Dates - matching,Dates - OWID greater,Dates - OWID lesser,Total Vax - matching,Total Vax - matching- WHO total,Total Vax - matching- OWID total,Total Vax - OWID greater - count,Total Vax - OWID greater - WHO total,Total Vax - OWID greater - OWID total,Total Vax - OWID greater - Diff,Total Vax - OWID lesser - count,Total Vax - OWID lesser - WHO total,Total Vax - OWID lesser - OWID total,Total Vax - OWID lesser - Diff,Total Vax - WHO total,Total Vax - OWID total,Total Vax - Diff
0,2022-10-03 14:24:10.561447-04:00,214,103,82,29,110,2586464480,2586464480,73,5137235433,5202839257,65603824,31,4942401242,4875695027,66706215,12666101155,12664998764,1102391


# Isolate anomalies

In [20]:
# OWID source is WHO but OWID total vaccinations do not match WHO
og = merged_data.loc[
          (merged_data["OWID Vax Source Category"] == 'World Health Organization') &
          (merged_data.diff_total_vaccinations!=0) 
].drop(columns=['people_fully_vaccinated','total_boosters','COUNTRY','ISO3','DATA_SOURCE','Code','OWID Vax Source Category']).sort_values(by=['diff_total_vaccinations'])
print('found mismatches: ', len(og))
og

found mismatches:  4


,iso_code,owid_date,location,owid_total_vaccinations,WHO_DATE,WHO_TOTAL_VACCINATIONS,diff_total_vaccinations
130,MOZ,2022-09-18,Mozambique,24676660.0,2022-09-18,19127825.0,-5548835.0
194,TUN,2022-09-28,Tunisia,14827155.0,2022-09-28,13146062.0,-1681093.0
165,SEN,2022-09-18,Senegal,2469499.0,2022-09-18,2165205.0,-304294.0
114,LSO,2022-07-17,Lesotho,1077116.0,2022-08-14,1102069.0,24953.0


# Inferring WHO sources

In [21]:
def inferred_source(owid_source, who_source, diff_total_vaccinations):
  if ((owid_source == 'Country') or ((owid_source == 'Other'))) and (diff_total_vaccinations==0):
    return "Maybe Public Reporting"
  elif (owid_source == 'Country') and (who_source == 'REPORTING')  and (diff_total_vaccinations!=0):
    return "Direct Reporting to WHO"
  else:
    return 'Unknown'

merged_data['who_inferred_source'] = merged_data.apply(lambda x: inferred_source(x['OWID Vax Source Category'], x['DATA_SOURCE'], x['diff_total_vaccinations']), axis=1)
merged_data

,iso_code,owid_date,location,owid_total_vaccinations,people_fully_vaccinated,total_boosters,COUNTRY,ISO3,DATA_SOURCE,WHO_DATE,WHO_TOTAL_VACCINATIONS,Code,OWID Vax Source Category,diff_total_vaccinations,who_inferred_source
0,ABW,2022-09-30,Aruba,173442.0,83617.0,NaN,Aruba,ABW,REPORTING,2022-09-16,173306.0,ABW,Country,-136.0,Direct Reporting to WHO
1,AFG,2022-09-28,Afghanistan,11881035.0,10245183.0,NaN,Afghanistan,AFG,REPORTING,2022-09-28,11881035.0,AFG,World Health Organization,0.0,Unknown
2,AGO,2022-09-04,Angola,22297104.0,7814121.0,1115824.0,Angola,AGO,REPORTING,2022-09-04,22297104.0,AGO,World Health Organization,0.0,Unknown
3,AIA,2022-09-30,Anguilla,24408.0,10342.0,3216.0,Anguilla,AIA,REPORTING,2022-09-16,24102.0,AIA,Pan American Health Organization,-306.0,Unknown
4,ALB,2022-09-18,Albania,2974247.0,1262982.0,356077.0,Albania,ALB,REPORTING,2022-09-18,2974247.0,ALB,World Health Organization,0.0,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,WSM,2022-09-26,Samoa,448009.0,177489.0,78510.0,Samoa,WSM,REPORTING,2022-09-12,510313.0,WSM,Pacific Data Hub (PDH),62304.0,Unknown
210,YEM,2022-09-28,Yemen,1065395.0,604202.0,33507.0,Yemen,YEM,REPORTING,2022-09-28,1065395.0,YEM,World Health Organization,0.0,Unknown
211,ZAF,2022-09-30,South Africa,37641904.0,19401052.0,3738814.0,South Africa,ZAF,REPORTING,2022-09-18,37580608.0,ZAF,Country,-61296.0,Direct Reporting to WHO
212,ZMB,2022-09-28,Zambia,9408734.0,6096991.0,33684.0,Zambia,ZMB,REPORTING,2022-09-04,9015082.0,ZMB,Country,-393652.0,Direct Reporting to WHO


In [26]:
import altair as alt
alt.Chart(merged_data).mark_bar().encode(
    y=alt.Y('who_inferred_source'),
    x='count()',
    tooltip='count()'
)

alt.Chart(...)